# Cryptostock Analytica

In [ ]:
# Initial imports
import os
import timeS
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import hvplot.pandas
import holoviews as hv
import numpy as np

%matplotlib inline

### Function definitions

In [ ]:
# Fetch stock data for a given stock symbol and date range
def fetch_stock_data(stock_symbol, start_date, end_date):
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], utc=True)
    return stock_data

# Fetch the top N cryptocurrencies by market cap
def fetch_top_n_cryptos(n=10):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': n,
        'page': 1,
        'sparkline': False,
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

# Fetch historical price data for a specific cryptocurrency using its coin_id
def fetch_crypto_data(coin_id, from_timestamp, to_timestamp):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': from_timestamp,
        'to': to_timestamp
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['prices'], columns=['time', 'price'])
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    return df

# Get the historical data of a specific cryptocurrency by its name
def get_data_by_name(name):
    for crypto in top_cryptos:
        if crypto['name'].lower() == name.lower():
            coin_id = crypto['id']
            return crypto_data[coin_id]
    return None

# Calculate the Pearson correlation between two datasets
def pearson_correlation(crypto, stocks):
    return pd.Series(crypto).corr(pd.Series(stocks))

# Interpret the correlation value
def interpret_correlation(correlation):
    if correlation >= 0.7:
        interpretation = "strong positive correlation"
    elif correlation >= 0.3:
        interpretation = "moderate positive correlation"
    elif correlation >= -0.3:
        interpretation = "weak correlation"
    elif correlation >= -0.7:
        interpretation = "moderate negative correlation"
    else:
        interpretation = "strong negative correlation"
    return interpretation

# Define a function to calculate Simple Moving Averages (SMA)
def calculate_sma(data, window):
    return data.rolling(window=window).mean()

# Define a function to calculate the Relative Strength Index (RSI)
def calculate_rsi(data, window):
    delta = data.diff().dropna()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    avg_gain = gains.rolling(window=window).mean()
    avg_loss = losses.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Define a function to calculate Bollinger Bands
def calculate_bollinger_bands(data, window, num_std):
    sma = data.rolling(window=window).mean()
    std = data.rolling(window=window).std()
    upper_band = sma + (num_std * std)
    lower_band = sma - (num_std * std)
    return sma, upper_band, lower_band

# Define a function to calculate Value at Risk (VaR) for a given confidence level
def calculate_var(returns, confidence_level=0.95):
    return returns.quantile(1 - confidence_level)

### Data Acquisition

In [ ]:
# Stock data acquisition
# Ask user for the stock symbol
selected_stock = input("Enter the stock symbol you want to view data for: ").upper()

end_date = pd.Timestamp.now(tz='UTC')
start_date = end_date - pd.DateOffset(years=3)

# Fetch the stock data
selected_stock_data = fetch_stock_data(selected_stock, start_date, end_date)

# Print the stock data
if not selected_stock_data.empty:
    # Round the price data to 2 decimal places
    selected_stock_data[['Open', 'High', 'Low', 'Close']] = selected_stock_data[['Open', 'High', 'Low', 'Close']].round(2)

    # Print the data
    print(selected_stock_data)
else:
    print("Error: Invalid stock symbol or no data available.")

selected_stock_data.set_index('Date')



In [ ]:
# Cryptocurrency data acquisition
# Unix timestamps for three years of data
to_timestamp = pd.Timestamp.now(tz='UTC').timestamp()
from_timestamp = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=3)
from_timestamp = from_timestamp.timestamp()

# Fetch the top 10 cryptocurrencies by market cap
top_cryptos = fetch_top_n_cryptos(10)

# Fetch historical data for each of the top 10 cryptos
crypto_data = {}
for crypto in top_cryptos:
    coin_id = crypto['id']
    try:
        df = fetch_crypto_data(coin_id, from_timestamp, to_timestamp)
        crypto_data[coin_id] = df
        print(f"Fetched data for {coin_id}")
        time.sleep(2)  # Add a delay between requests to avoid rate limiting
    except Exception as e:
        print(f"Error fetching data for {coin_id}: {e}")
        
# Ask user for the name of the cryptocurrency
selected_crypto = input("Enter the name of the cryptocurrency you want to view data for: ")

# Get the data for the selected cryptocurrency
selected_crypto_data = get_data_by_name(selected_crypto)
    
# Print the cryptocurrency data
if selected_crypto_data is not None:
    # Round the price data to 2 decimal places
    selected_crypto_data['price'] = selected_crypto_data['price'].round(2)

    # Print the data
    print(selected_crypto_data)
else:
    print("Error: Invalid cryptocurrency name.")


### Reusable Data Frames #ESTO ES NUEVO

In [ ]:
##### Bloque completo es nuevo ######

### Closing Prices for choosen assets merged on same dates ###

# Mergin the date frame on the date 
selected_stock_data['Date'] = selected_stock_data['Date'].dt.tz_convert(None)
Combined_prices_db = pd.merge(selected_crypto_data, selected_stock_data, left_on='time', right_on='Date', how='inner')

# cleaning data by dropping values that we are not needed 

Combined_prices_db = Combined_prices_db.drop(columns= ['Open', 'time','Open', 'High', 'Low', 'Adj Close', 'Volume'])


# Final data frame with the closing prices and date as Index and reanaming columns with their corresponfing asset

Combined_prices_db.set_index('Date', inplace= True)
Combined_prices_db.rename(columns={'price': selected_crypto, 'Close': selected_stock}, inplace=True)

Combined_prices_db


Creating a DF combing closing prices where the dates match

In [ ]:
##### Bloque completo es nuevo ######


### Daily percent Change for both assets Data Frame

Stock_cryoto_pct = Combined_prices_db.pct_change().dropna()
Stock_cryoto_pct

 ## Part 1 - Correlation Analysis # Tiene cambios

In [ ]:
# Calculate the correlation between crypto and stock prices
correlation = pearson_correlation(selected_crypto_data['price'], selected_stock_data['Close'])
print(correlation)

# Create a scatter plot using Hvplot 

### Modifique el bloque completo para que use el data frame que cree arriba con el nombre de los activos selccionados ###

scatter_plot =  Combined_prices_db.hvplot.scatter(
    x=selected_crypto,
    y=selected_stock,
    title='Scatter Plot of Crypto Prices vs. Stock Prices'
)

# Position the correlation label on the scatter plot
x_position = 0.85 * Combined_prices_db[selected_crypto].max()
y_position = 0.6 * Combined_prices_db[selected_stock].max()
correlation_label = hv.Text(x_position , y_position , f'Correlation: {correlation:.2f}', fontsize=14)

# Get the textual interpretation of the correlation
correlation_interpretation = interpret_correlation(correlation)
print(f"The assets have a {correlation_interpretation}.")

# Position the interpretation label on the scatter plot
interpretation_label = hv.Text(x_position, y_position - y_position * 0.1, f'Interpretation: {correlation_interpretation}', fontsize=12)

# Combine the scatter plot, correlation label, and interpretation label
plot_with_label_and_interpretation = scatter_plot * correlation_label * interpretation_label

# Display the plot
plot_with_label_and_interpretation.opts(legend_position='top_left', height=400, width=900)

## Part 2 - Technical Analysis ## no toque esto

### Simple Moving Averages (SMA) ## no toque esto

In [ ]:
# Calculate 30-day and 100-day SMA for the selected cryptocurrency
selected_crypto_data['SMA30'] = calculate_sma(selected_crypto_data['price'], 30)
selected_crypto_data['SMA100'] = calculate_sma(selected_crypto_data['price'], 100)

# Calculate 30-day and 100-day SMA for the selected stock
selected_stock_data['SMA30'] = calculate_sma(selected_stock_data['Close'], 30)
selected_stock_data['SMA100'] = calculate_sma(selected_stock_data['Close'], 100)

# Create Simple Moving Averages (SMA) plot for the selected cryptocurrency
sma_crypto_plot = selected_crypto_data[['price', 'SMA30', 'SMA100']].hvplot(title=f'Simple Moving Averages for {selected_crypto}', ylabel='Price')

# Create Simple Moving Averages (SMA) plot for the selected stock
sma_stock_plot = selected_stock_data[['Close', 'SMA30', 'SMA100']].hvplot(title=f'Simple Moving Averages for {selected_stock}', ylabel='Price')

# Display the plots
(sma_crypto_plot + sma_stock_plot)

In [ ]:
### Relative Strength Index (RSI) ## No toque esto

In [ ]:
# Calculate the 14-day RSI for the selected cryptocurrency
selected_crypto_data['RSI'] = calculate_rsi(selected_crypto_data['price'], 14)

# Calculate the 14-day RSI for the selected stock
selected_stock_data['RSI'] = calculate_rsi(selected_stock_data['Close'], 14)

# Create Relative Strength Index (RSI) plot for the selected cryptocurrency
rsi_crypto_plot = selected_crypto_data['RSI'].hvplot(title=f'Relative Strength Index for {selected_crypto}', ylabel='RSI', ylim=(0, 100))

# Create Relative Strength Index (RSI) plot for the selected stock
rsi_stock_plot = selected_stock_data['RSI'].hvplot(title=f'Relative Strength Index for {selected_stock}', ylabel='RSI', ylim=(0, 100))

# Display the plots
(rsi_crypto_plot + rsi_stock_plot)

In [ ]:
### Bollinger Bands ## no toque esto

In [ ]:
# Calculate the Bollinger Bands for the selected cryptocurrency with a 20-day window and 2 standard deviations
selected_crypto_data['SMA20'], selected_crypto_data['UpperBB'], selected_crypto_data['LowerBB'] = calculate_bollinger_bands(selected_crypto_data['price'], 20, 2)

# Calculate the Bollinger Bands for the selected stock with a 20-day window and 2 standard deviations
selected_stock_data['SMA20'], selected_stock_data['UpperBB'], selected_stock_data['LowerBB'] = calculate_bollinger_bands(selected_stock_data['Close'], 20, 2)

# Create Bollinger Bands plot for the selected cryptocurrency
bb_crypto_plot = selected_crypto_data[['price', 'SMA20', 'UpperBB', 'LowerBB']].hvplot(title=f'Bollinger Bands for {selected_crypto}', ylabel='Price')

# Create Bollinger Bands plot for the selected stock
bb_stock_plot = selected_stock_data[['Close', 'SMA20', 'UpperBB', 'LowerBB']].hvplot(title=f'Bollinger Bands for {selected_stock}', ylabel='Price')

# Display the plots

(bb_crypto_plot + bb_stock_plot)

## Part 3 - Risk Analysis

### Standard Deviation ### modifique esto

In [ ]:
### Cambie el bloque completo 

# Calculate the annualized standard deviation for the cryptocurrency and stock
crypto_std_annualized = np.sqrt(252) * Stock_cryoto_pct[selected_crypto].std()
stocks_std_annualized = np.sqrt(252) * Stock_cryoto_pct[selected_stock].std()

# Create a DataFrame of the annualized standard deviation for the cryptocurrency and stock
assets_annualized_std = pd.DataFrame({
    "Asset type" : [ selected_crypto , selected_stock],
    "Annualized Standard Deviation": [crypto_std_annualized , stocks_std_annualized]
})

# Create a bar plot of the annualized standard deviation for the cryptocurrency and stock using Hvplot
assets_annualized_std.hvplot.bar(
    title= "Annualized Standard Deviation",
    x="Asset type",
    y="Annualized Standard Deviation"
)

Creating a data frame with the percent change for the assets

In [ ]:
##### modifique el bloque completo

Rolling_21_day_std = Stock_cryoto_pct.rolling(window= 21).std() * np.sqrt(21)

std_plot = Rolling_21_day_std.hvplot(
    title="Rolling 21-Day Standard Deviation",
    width=900,
    height=400,
    stacked=False,
    color=['blue', 'orange'],
   
    ylabel="Standard Deviation",

    ylim=(0, Rolling_21_day_std.max().max() * 1.1),
)

std_plot

### Value at Risk (VaR) ### Modique esto 

In [ ]:
#### los cambios que hice arriba afectaron esta parte, por esto tuve que modifiacarlo por igual

######borrar esta parte de aqui ###########################################################################
#### esto fue lo que modique , ahora el resultado nos dara el nombre del crypto y stock que escogimos ######
# # Calculate the 95% Value at Risk for the selected cryptocurrency
# crypto_var_95 = calculate_var(stock_and_crypto_returns_df['Crypto Prices'])
############################################################################################################

### Esta parte es la que usaramos


# Calculate the 95% Value at Risk for the selected cryptocurrency
crypto_var_95 = calculate_var(Stock_cryoto_pct[selected_crypto])

# Calculate the 95% Value at Risk for the selected stock
stocks_var_95 = calculate_var(Stock_cryoto_pct[selected_stock])

# Print the calculated 95% Value at Risk for both the cryptocurrency and stock
print(f"The 95% Value at Risk for the selected cryptocurrency is {crypto_var_95:.4f}")
print(f"The 95% Value at Risk for the selected stock is {stocks_var_95:.4f}")

# Create a DataFrame with the 95% Value at Risk for both the cryptocurrency and stock
var_df = pd.DataFrame({
    "Asset type" : [selected_crypto , selected_stock],
    "95% Value at Risk": [crypto_var_95, stocks_var_95]
})

# Create a bar plot of the 95% Value at Risk for both the cryptocurrency and stock using Hvplot
var_plot = var_df.hvplot.bar(
    title= "95% Value at Risk",
    x="Asset type",
    y="95% Value at Risk",
    ylim=(0, max(crypto_var_95, stocks_var_95) * 1.5)
)

# Display the plot
var_plot


In [ ]:
#### lo mismo que arriba, cambie por completo este bloque para que funcione

# Create a DataFrame with the 95% Value at Risk for both the cryptocurrency and stock
var_df = pd.DataFrame ({
     "Asset type" : [selected_crypto , selected_stock],
    "95% Value at Risk": [crypto_var_95, stocks_var_95]
    })

# Create a bar plot of the 95% Value at Risk for both the cryptocurrency and stock using Hvplot
var_plot = var_df.hvplot.bar(
    title= "95% Value at Risk",
    x="Asset type",
    y="95% Value at Risk",
    ylim=(0, max(crypto_var_95, stocks_var_95) * 1.5)
)

# Display the plot
var_plot
